In [1]:
#%% Imports:
import matplotlib
matplotlib.use('MACOSX')

from math import sqrt
import os
import numpy as np
import mne
#import matplotlib #doesnt it import through mne already?
from copy import deepcopy

#Open data:
#sample_data_folder = mne.datasets.sample.data_path()
#kath_raw_file2 = "/Users/jenya/Documents/Oldenburg and university/Job Uni Rieger lab/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif"
#kath_raw_file = os.path.join('Katharinas_Data','sub_HT05ND16', '210811', 'mikado-1.fif')
kath_raw_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif'
print(kath_raw_file)
#print(kath_raw_file2)                                   
raw = mne.io.read_raw_fif(kath_raw_file)
#raw.crop(0, 60).load_data()  # just use a fraction of data for speed here

#Print info about the data:
print(raw)
print(raw.info)
raw

#crop the data to calculate faster

raw_cropped = raw.copy()
raw_cropped.crop(0, 300) #(first 5 min)

/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 1809000 ... 3375999 =   1809.000 ...  3375.999 secs
Ready.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-2.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif

/var/folders/h2/khhmb4p510vg63hbv0qkftt80000gs/T/ipykernel_28548/42220836.py:19: RuntimeWarning: This filename (/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(kath_raw_file)


<Raw | mikado-1.fif, 319 x 3390000 (3390.0 s), ~6.6 MB, data not loaded>
<Info | 24 non-empty values
 acq_pars: ACQactiveGround 1 ACQch.BIO001.gain 2000 ACQch.BIO001.highpass ...
 bads: []
 ch_names: IASX+, IASX-, IASY+, IASY-, IASZ+, IASZ-, IAS_DX, IAS_DY, IAS_X, ...
 chs: 11 IAS, 102 Magnetometers, 204 Gradiometers, 1 Stimulus, 1 SYST
 custom_ref_applied: False
 description: TRIUX system at Oldenburg_3127
 dev_head_t: MEG device -> head transform
 dig: 356 items (3 Cardinal, 5 HPI, 348 Extra)
 events: 1 item (list)
 experimenter: Meg User (meg)
 file_id: 4 items (dict)
 gantry_angle: 68.0
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 hpi_subsystem: 3 items (dict)
 line_freq: 50.0
 lowpass: 330.0 Hz
 meas_date: 2021-08-11 12:05:17 UTC
 meas_id: 4 items (dict)
 nchan: 319
 proj_id: 1 item (ndarray)
 proj_name: mikado
 projs: magn8_iasoff_68deg.fif : PCA-v1: off, magn8_iasoff_68deg.fif : ...
 sfreq: 1000.0 Hz
 subject_info: 6 items (dict)
>


<Raw | mikado-1.fif, 319 x 300001 (300.0 s), ~6.6 MB, data not loaded>

In [2]:
#%% Find magnetometers and gradiometers:
# unit t - magenetometer. unit M_T - gradiometer. (in this set name will end with 1 for magnet, 
# with 2and3 for grad.)

mags = []
grads = []

for i, chs in enumerate(raw_cropped.info['chs']):

    if str(chs['unit']).endswith('UNIT_T)'):
        mags.append((chs['ch_name'], i))
    elif str(chs['unit']).endswith('UNIT_T_M)'):
        grads.append((chs['ch_name'], i))

In [3]:
#epoch channels together

events = mne.find_events(raw_cropped, stim_channel='STI101', min_duration=1.2)
epochs = mne.Epochs(raw_cropped, events, tmin=-0.2, tmax=1, preload=True, baseline = None) #, event_id=event_dict, reject=reject_criteria)
df = epochs.to_data_frame(time_format=None, scalings=dict(mag=1, grad=1))

#by default, channel measurement values are scaled so that EEG data are converted 
# to µV, magnetometer data are converted to fT, and gradiometer data are converted 
# to fT/cm. These scalings can be customized through the scalings parameter, or 
# suppressed by passing scalings=dict(eeg=1, mag=1, grad=1).
df



Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
8 events found
Event IDs: [ 9 19 20 21 22]
Not setting metadata
Not setting metadata
8 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 8)
8 projection items activated
Loading data for 8 events and 1201 original time points ...
0 bad epochs dropped


/var/folders/h2/khhmb4p510vg63hbv0qkftt80000gs/T/ipykernel_28548/3422256896.py:3: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  events = mne.find_events(raw_cropped, stim_channel='STI101', min_duration=1.2)


,time,condition,epoch,IASX+,IASX-,IASY+,IASY-,IASZ+,IASZ-,IAS_DX,...,MEG2622,MEG2623,MEG2631,MEG2632,MEG2633,MEG2641,MEG2642,MEG2643,STI101,SYS201
0,-0.200,19,0,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.095108e-29,-5.180752e-29,-1.192093e-06,...,1.194763e-12,7.765961e-12,-1.547003e-13,-9.358978e-12,-7.367706e-12,-7.218119e-14,-1.373978e-11,6.571197e-12,18.0,0.0
1,-0.199,19,0,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,...,1.015549e-11,-5.575562e-12,-8.445258e-14,-4.779053e-12,-6.571197e-12,4.878084e-14,-5.575562e-12,8.960724e-12,18.0,0.0
2,-0.198,19,0,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,...,5.575562e-12,-1.533279e-11,-1.631808e-14,-8.363342e-12,-1.373978e-11,1.435712e-13,-3.783417e-12,7.765961e-12,18.0,0.0
3,-0.197,19,0,7.974282e-29,-1.192093e-06,-9.600046e-31,2.019014e-29,-1.192093e-06,-1.192093e-06,7.317702e-30,...,1.991272e-12,1.991272e-12,-9.371890e-14,-1.095200e-11,-1.294327e-11,2.456428e-15,-7.765961e-12,5.973816e-13,18.0,0.0
4,-0.196,19,0,7.337872e-29,7.717356e-29,5.197823e-30,1.919949e-29,-1.581347e-29,-7.481316e-29,8.938612e-30,...,9.159851e-12,1.413803e-11,-1.693029e-13,-1.593018e-12,-1.991272e-13,-1.668383e-13,1.991272e-12,4.978180e-12,18.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9603,0.996,20,7,-1.192093e-06,-1.192093e-06,-1.683182e-29,6.909267e-31,1.010209e-30,1.681756e-29,-1.192093e-06,...,7.566833e-12,5.973816e-13,-1.825679e-13,5.575562e-12,6.571197e-12,4.224788e-14,-5.973816e-12,9.956360e-12,20.0,0.0
9604,0.997,20,7,-1.192093e-06,4.738726e-30,-1.192093e-06,-1.192093e-06,-1.358453e-30,-1.885637e-30,-1.192093e-06,...,1.334152e-11,3.982544e-12,-1.486202e-13,6.172943e-12,0.000000e+00,-2.174185e-14,-7.765961e-12,9.956360e-13,20.0,0.0
9605,0.998,20,7,-1.192093e-06,-1.726060e-29,-1.192093e-06,-1.192093e-06,9.504517e-30,3.352403e-29,-1.192093e-06,...,9.757233e-12,-1.792145e-12,-8.367578e-14,-1.991272e-13,-1.194763e-12,-3.944463e-15,-1.433716e-11,-3.783417e-12,20.0,0.0
9606,0.999,20,7,9.138198e-30,-1.909005e-29,-1.192093e-06,-1.192093e-06,1.661232e-29,5.756178e-29,-1.192093e-06,...,9.757233e-12,-2.588654e-12,-1.883981e-13,-7.168579e-12,-1.991272e-12,5.192944e-14,-2.011185e-11,-3.982544e-13,20.0,0.0


In [5]:
df['MEG0111'][0]

-1.0525499934360496e-13